In [2]:
import pandas as pd


In [9]:
test_df = pd.read_csv('data/german/test.csv')
train_df = pd.read_csv('data/german/train.csv')

test_df.head()



,text,label,source
0,@user korrekt! Verstehe sowas nicht...,negative,sb_10k
1,Einparken können die Aliens auch nicht! #schlefaz,neutral,sb_10k
2,Der Dubbletimepart von Julien war ja mal sowas...,positive,sb_10k
3,#Instachallenge #Day16 #what #i #am #reading #...,negative,sb_10k
4,Also gleich. Mach noch das Video fertig.,neutral,sb_10k


In [12]:
train_df['label'].unique()


mapping = {'negative': 0, 'neutral': 1, 'positive': 2}

# Create a new column with mapped values
train_df['sentiment_num'] = train_df['label'].map(mapping)
test_df['sentiment_num'] = test_df['label'].map(mapping)

In [13]:
test_df.head()

,text,label,source,sentiment_num
0,@user korrekt! Verstehe sowas nicht...,negative,sb_10k,0
1,Einparken können die Aliens auch nicht! #schlefaz,neutral,sb_10k,1
2,Der Dubbletimepart von Julien war ja mal sowas...,positive,sb_10k,2
3,#Instachallenge #Day16 #what #i #am #reading #...,negative,sb_10k,0
4,Also gleich. Mach noch das Video fertig.,neutral,sb_10k,1


In [20]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, AdamW
from datasets import Dataset

model_name = "bert-base-multilingual-uncased"
tok = BertTokenizer.from_pretrained(model_name)

def tokenize_function (examples):
    return tok(examples['text'],padding = "max_length", truncation = True, max_length=128)

train_dataset = Dataset.from_pandas(train_df) 

tokenized_train = train_dataset.map(tokenize_function, batched=True)

tokenized_train = tokenized_train.rename_column("sentiment_num", "labels")
tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

print(tokenized_train[0])

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'is_mlu_available' from 'accelerate.utils' (/opt/miniconda3/envs/mbertproj/lib/python3.11/site-packages/accelerate/utils/__init__.py)

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, shuffle = True, batch_size = 8)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
